# Biomedical LLM Evaluation Suite

**Project:** Pipeline Optimisation
**Purpose:** Evaluate language models on literature interpretation tasks

In [ ]:
# Login to HuggingFace (only needed for gated models like Meditron, Llama)
# Get your token from: https://huggingface.co/settings/tokens

from huggingface_hub import login

# Option 1: Interactive login (recommended for Colab)
login()

# Option 2: Login with token directly (uncomment and add your token)
# login(token='hf_YOUR_TOKEN_HERE')

print('✓ Logged in to HuggingFace')

In [ ]:
import json
import time
from pathlib import Path
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
import torch
from tqdm.notebook import tqdm

print('✓ Imports successful')
print(f"Device: {'GPU (CUDA)' if torch.cuda.is_available() else 'CPU'}")
print(f"PyTorch version: {torch.__version__}")

In [ ]:
# Load test datasets
test_dir = Path('model_tests')

with open(test_dir / 'test_relevance.json') as f:
    relevance_data = json.load(f)

with open(test_dir / 'test_mechanism.json') as f:
    mechanism_data = json.load(f)

with open(test_dir / 'test_quality.json') as f:
    quality_data = json.load(f)

with open(test_dir / 'test_stability.json') as f:
    stability_data = json.load(f)

print(f'✓ Loaded test data:')
print(f'  - Relevance: {len(relevance_data)} items')
print(f'  - Mechanism: {len(mechanism_data)} items')
print(f'  - Quality: {len(quality_data)} items')
print(f'  - Stability: {len(stability_data)} items')

In [ ]:
# Preview a test item
sample = relevance_data[0]
print(f"ID: {sample['id']}")
print(f"Agent: {sample['agent']}")
print(f"Pathway: {sample['pathway']}")
print(f"Gold Label: {sample['gold_label']}")
print(f"\nAbstract (first 200 chars):\n{sample['abstract'][:200]}...")

In [ ]:
class ModelEvaluator:
    def __init__(self, model_name, device='auto'):
        self.model_name = model_name
        self.device = device if device != 'auto' else ('cuda' if torch.cuda.is_available() else 'cpu')
        
        print(f'Loading model: {model_name}')
        print(f'Device: {self.device}')
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        
        try:
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                trust_remote_code=True,
                torch_dtype=torch.float16 if self.device == 'cuda' else torch.float32,
                device_map=self.device if self.device == 'cuda' else None
            )
            self.is_causal = True
        except:
            self.model = AutoModelForSeq2SeqLM.from_pretrained(
                model_name,
                trust_remote_code=True,
                torch_dtype=torch.float16 if self.device == 'cuda' else torch.float32,
                device_map=self.device if self.device == 'cuda' else None
            )
            self.is_causal = False
        
        if self.device == 'cpu':
            self.model = self.model.to(self.device)
        
        self.model.eval()
        
        # Fix tokenizer pad token - don't set it to eos_token
        if self.tokenizer.pad_token is None:
            # For models like Mistral/BioMistral, use unk_token or add a new pad token
            if self.tokenizer.unk_token is not None:
                self.tokenizer.pad_token = self.tokenizer.unk_token
            else:
                # Add a new pad token
                self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
                self.model.resize_token_embeddings(len(self.tokenizer))
        
        if hasattr(self.model.config, 'max_position_embeddings'):
            self.model_max_length = self.model.config.max_position_embeddings
        elif hasattr(self.model.config, 'n_positions'):
            self.model_max_length = self.model.config.n_positions
        else:
            self.model_max_length = 1024
        
        print(f'✓ Model loaded (max length: {self.model_max_length})')
        print(f'  Tokenizer vocab size: {len(self.tokenizer)}')
        print(f'  Tokenizer pad token: {self.tokenizer.pad_token} (ID: {self.tokenizer.pad_token_id})')
        print(f'  Tokenizer eos token: {self.tokenizer.eos_token} (ID: {self.tokenizer.eos_token_id})')
        print(f'  Model vocab size: {self.model.config.vocab_size}')
    
    def generate_response(self, prompt, max_new_tokens=256):
        safe_input_length = self.model_max_length - max_new_tokens - 10
        
        inputs = self.tokenizer(
            prompt, 
            return_tensors='pt', 
            truncation=True, 
            max_length=safe_input_length,
            padding=False  # Don't pad single sequences
        )
        
        # Move to device and remove unnecessary keys
        inputs = {k: v.to(self.device) for k, v in inputs.items() if k in ['input_ids', 'attention_mask']}
        
        input_length = inputs['input_ids'].shape[1]
        
        try:
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    min_new_tokens=20,  # Force minimum generation
                    temperature=0.7,  # Increased for better sampling
                    do_sample=True,
                    top_p=0.9,
                    repetition_penalty=1.1,
                    pad_token_id=self.tokenizer.pad_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                )
            
            # Extract only the generated tokens (not the input)
            generated_tokens = outputs[0][input_length:]
            response = self.tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
            
            # Debug: print first 200 chars if response is empty or very short
            if len(response) < 10:
                full_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                print(f"[DEBUG] Short response detected!")
                print(f"[DEBUG] Input length: {input_length} tokens")
                print(f"[DEBUG] Output length: {outputs[0].shape[0]} tokens")
                print(f"[DEBUG] Generated tokens: {len(generated_tokens)}")
                print(f"[DEBUG] Response: '{response}'")
                print(f"[DEBUG] Full text (first 300 chars): {full_text[:300]}")
            
            return response if response else "No response generated"
        except Exception as e:
            import traceback
            error_msg = f'{{"error": "Generation failed: {str(e)}"}}'
            print(f"[ERROR] {error_msg}")
            print(traceback.format_exc())
            return error_msg
    
    def extract_json(self, response):
        try:
            if '```json' in response:
                start = response.find('```json') + 7
                end = response.find('```', start)
                json_str = response[start:end].strip()
            elif '{' in response:
                start = response.find('{')
                end = response.rfind('}') + 1
                json_str = response[start:end]
            else:
                json_str = response
            
            return json.loads(json_str)
        except Exception as e:
            return {'error': f'JSON parsing failed: {str(e)}', 'raw_response': response}

print('✓ ModelEvaluator class defined')

## Load Model

**Recommended models:**
- `facebook/galactica-1.3b` - Best for Colab free tier
- `facebook/galactica-125m` - Fastest, may struggle
- `BioMistral/BioMistral-7B` - Best accuracy, needs Colab Pro

In [ ]:
# Change model here
MODEL_NAME = 'facebook/galactica-1.3b'

evaluator = ModelEvaluator(MODEL_NAME)

## Quick Test (2 items)

Test with 2 items first to verify the model works

In [ ]:
prompt_template = """You are a biomedical expert analyzing scientific literature.

Task: Does this abstract explain how {agent} affects {pathway}?

Agent: {agent}
Pathway: {pathway}

Abstract:
{abstract}

Instructions:
Read the abstract carefully. You must determine if it describes the molecular mechanism connecting THIS SPECIFIC agent to THIS SPECIFIC pathway.

Answer "relevant" ONLY if the abstract:
✓ Names or describes {pathway} explicitly (or its key components)
✓ Explains how {agent} molecularly interacts with {pathway}
✓ Includes specific proteins/genes showing the connection

Answer "not_relevant" if:
✗ The abstract discusses {agent} but talks about a different pathway
✗ The abstract discusses {pathway} but not in relation to {agent}  
✗ The agent and pathway are not connected in the text
✗ Only clinical outcomes mentioned, no molecular mechanism

In your rationale:
- State which pathway is actually discussed in the abstract
- State whether that matches {pathway}
- Be specific about what is or isn't connected

ONLY respond in following JSON output format. DO NOT add or remove any values. Anything written inbetween square brackets [] is a placeholder and should be replaced. Do not add anything other than the JSON output in your response

Output format:
{{
  "relevance": "relevant" or "not_relevant",
  "rationale": "The abstract discusses [agent]'s effect on [actual pathway mentioned]. This [does/does not] match the specified pathway ({pathway})."
}}

JSON Response:"""

quick_test = relevance_data[:2]
results = []

for item in tqdm(quick_test, desc='Quick Test'):
    prompt = prompt_template.format(
        agent=item['agent'],
        pathway=item['pathway'],
        abstract=item['abstract']
    )
    
    start = time.time()
    response = evaluator.generate_response(prompt)
    elapsed = time.time() - start
    
    parsed = evaluator.extract_json(response)
    predicted = parsed.get('relevance', '').lower()
    expected = item['gold_label'].lower()
    
    results.append({
        'id': item['id'],
        'expected': expected,
        'predicted': predicted,
        'correct': predicted == expected,
        'time': elapsed
    })
    
    print(f"{item['id']}: {predicted} (expected: {expected}) - {'✓' if predicted == expected else '✗'} [{elapsed:.1f}s]")
    print(f"Response: {response[:150]}...\n")

accuracy = sum(r['correct'] for r in results) / len(results)
print(f"\nQuick Test Accuracy: {accuracy:.1%}")
print(f"Avg Time: {sum(r['time'] for r in results)/len(results):.1f}s per item")

## Test 1: Relevance Assessment (10 items)

In [ ]:
relevance_results = []

for item in tqdm(relevance_data, desc='Relevance Test'):
    prompt = prompt_template.format(
        agent=item['agent'],
        pathway=item['pathway'],
        abstract=item['abstract']
    )
    
    start = time.time()
    response = evaluator.generate_response(prompt)
    elapsed = time.time() - start
    
    parsed = evaluator.extract_json(response)
    predicted = parsed.get('relevance', '').lower()
    expected = item['gold_label'].lower()
    
    relevance_results.append({
        'id': item['id'],
        'agent': item['agent'],
        'pathway': item['pathway'],
        'expected': expected,
        'predicted': predicted,
        'correct': predicted == expected,
        'time': elapsed
    })

df_relevance = pd.DataFrame(relevance_results)
display(df_relevance)

relevance_accuracy = df_relevance['correct'].mean()
print(f"\nRelevance Accuracy: {relevance_accuracy:.1%}")
print(f"Correct: {df_relevance['correct'].sum()}/{len(df_relevance)}")
print(f"Avg Time: {df_relevance['time'].mean():.1f}s per item")

## Test 2: Mechanism Extraction (5 items)

In [ ]:
mechanism_prompt = '''You are a biomedical expert extracting mechanistic information from scientific literature.

Task: Extract the mechanism by which {agent} affects {pathway}.

Agent: {agent}
Pathway: {pathway}

Abstract: {abstract}

Provide extraction as JSON:
{{
  "mechanism_summary": "brief description of how the agent affects the pathway",
  "molecular_components": ["list", "of", "key", "molecules/proteins/genes"],
  "direction_of_effect": "activation" or "inhibition" or "unknown"
}}

JSON Response:'''

mechanism_results = []

for item in tqdm(mechanism_data, desc='Mechanism Test'):
    prompt = mechanism_prompt.format(
        agent=item['agent'],
        pathway=item['pathway'],
        abstract=item['abstract']
    )
    
    start = time.time()
    response = evaluator.generate_response(prompt)
    elapsed = time.time() - start
    
    parsed = evaluator.extract_json(response)
    predicted = parsed.get('direction_of_effect', '').lower()
    expected = item['gold_label'].lower()
    
    # Check if molecular components were extracted
    components = parsed.get('molecular_components', [])
    has_components = len(components) > 0 if isinstance(components, list) else False
    
    mechanism_results.append({
        'id': item['id'],
        'expected': expected,
        'predicted': predicted,
        'correct': predicted == expected,
        'has_components': has_components,
        'num_components': len(components) if isinstance(components, list) else 0,
        'time': elapsed
    })

df_mechanism = pd.DataFrame(mechanism_results)
display(df_mechanism)

mechanism_accuracy = df_mechanism['correct'].mean()
component_rate = df_mechanism['has_components'].mean()
print(f"\nMechanism Direction Accuracy: {mechanism_accuracy:.1%}")
print(f"Correct: {df_mechanism['correct'].sum()}/{len(df_mechanism)}")
print(f"Component Extraction Rate: {component_rate:.1%}")
print(f"Avg Components Extracted: {df_mechanism['num_components'].mean():.1f}")

## Test 3: Evidence Quality (5 items)

In [ ]:
quality_prompt = '''You are a biomedical expert evaluating scientific evidence.

Task: Assess the strength of evidence in this abstract.

Evidence Quality Criteria:
- Strong: Multiple experimental approaches, rigorous controls, clinical validation
- Moderate: In vitro data with multiple assays
- Weak: Limited experimental data
- Insufficient: Minimal data, unclear methods

Agent: {agent}
Pathway: {pathway}

Abstract: {abstract}

Provide assessment as JSON:
{{
  "evidence_quality": "strong" or "moderate" or "weak" or "insufficient",
  "justification": "brief explanation"
}}

JSON Response:'''

quality_results = []

for item in tqdm(quality_data, desc='Quality Test'):
    prompt = quality_prompt.format(
        agent=item['agent'],
        pathway=item['pathway'],
        abstract=item['abstract']
    )
    
    start = time.time()
    response = evaluator.generate_response(prompt)
    elapsed = time.time() - start
    
    parsed = evaluator.extract_json(response)
    predicted = parsed.get('evidence_quality', '').lower()
    expected = item['gold_label'].lower()
    
    quality_results.append({
        'id': item['id'],
        'expected': expected,
        'predicted': predicted,
        'correct': predicted == expected,
        'time': elapsed
    })

df_quality = pd.DataFrame(quality_results)
display(df_quality)

quality_accuracy = df_quality['correct'].mean()
print(f"\nQuality Accuracy: {quality_accuracy:.1%}")
print(f"Correct: {df_quality['correct'].sum()}/{len(df_quality)}")

## Test 4: JSON Output Stability (5 items)

Run each prompt twice to check if the model produces consistent JSON structure.

In [ ]:
stability_prompt = '''You are a biomedical expert analyzing scientific literature.

Task: Does this abstract explain how {agent} affects {pathway}?

Agent: {agent}
Pathway: {pathway}

Abstract: {abstract}

Respond ONLY with this exact JSON structure:
{{
  "relevance": "relevant" or "not_relevant",
  "rationale": "brief explanation"
}}

JSON Response:'''

def check_json_stability(parsed1, parsed2):
    """Check if two parsed JSON responses have matching structure."""
    if 'error' in parsed1 or 'error' in parsed2:
        return False, 'json_parse_failed'
    
    # Check field names match
    keys1 = set(parsed1.keys())
    keys2 = set(parsed2.keys())
    if keys1 != keys2:
        return False, 'field_mismatch'
    
    # Check data types match
    for key in keys1:
        if type(parsed1[key]) != type(parsed2[key]):
            return False, 'type_mismatch'
    
    return True, 'stable'

stability_results = []

for item in tqdm(stability_data, desc='Stability Test'):
    prompt = stability_prompt.format(
        agent=item['agent'],
        pathway=item['pathway'],
        abstract=item['abstract']
    )
    
    # Run twice
    response1 = evaluator.generate_response(prompt)
    parsed1 = evaluator.extract_json(response1)
    
    response2 = evaluator.generate_response(prompt)
    parsed2 = evaluator.extract_json(response2)
    
    # Check stability
    is_stable, reason = check_json_stability(parsed1, parsed2)
    
    # Check for hallucinated fields (fields not in expected schema)
    expected_fields = {'relevance', 'rationale'}
    extra_fields1 = set(parsed1.keys()) - expected_fields - {'error', 'raw_response'}
    extra_fields2 = set(parsed2.keys()) - expected_fields - {'error', 'raw_response'}
    has_hallucination = len(extra_fields1) > 0 or len(extra_fields2) > 0
    
    stability_results.append({
        'id': item['id'],
        'stable': is_stable,
        'reason': reason,
        'has_hallucinated_fields': has_hallucination,
        'run1_fields': list(parsed1.keys()),
        'run2_fields': list(parsed2.keys())
    })

df_stability = pd.DataFrame(stability_results)
display(df_stability)

stability_rate = df_stability['stable'].mean()
hallucination_rate = df_stability['has_hallucinated_fields'].mean()
print(f"\nJSON Stability Rate: {stability_rate:.1%}")
print(f"Stable: {df_stability['stable'].sum()}/{len(df_stability)}")
print(f"Hallucination Rate: {hallucination_rate:.1%}")

## Summary and Visualization

In [ ]:
# Summary
print(f"{'='*80}")
print(f"EVALUATION SUMMARY: {MODEL_NAME}")
print(f"{'='*80}")
print(f"Test 1 - Relevance Accuracy:    {relevance_accuracy:.1%}")
print(f"Test 2 - Mechanism Accuracy:    {mechanism_accuracy:.1%}")
print(f"Test 3 - Evidence Quality:      {quality_accuracy:.1%}")
print(f"Test 4 - JSON Stability:        {stability_rate:.1%}")
print(f"Avg Inference Time:             {df_relevance['time'].mean():.1f}s per item")
print(f"{'='*80}")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Accuracy comparison
accuracies = {
    'Relevance': relevance_accuracy,
    'Mechanism': mechanism_accuracy,
    'Quality': quality_accuracy,
    'Stability': stability_rate
}
colors = ['#1f77b4', '#2ca02c', '#ff7f0e', '#9467bd']
axes[0].bar(accuracies.keys(), accuracies.values(), color=colors)
axes[0].set_ylabel('Accuracy / Rate')
axes[0].set_title('Performance by Test Type')
axes[0].set_ylim([0, 1])
axes[0].axhline(y=0.5, color='r', linestyle='--', alpha=0.5)

# Time distribution
all_times = (list(df_relevance['time']) + list(df_mechanism['time']) + 
             list(df_quality['time']))
axes[1].hist(all_times, bins=15, color='#1f77b4', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Time (seconds)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Inference Time Distribution')

plt.tight_layout()
plt.show()

# Save results
summary_df = pd.DataFrame([{
    'Model': MODEL_NAME,
    'Relevance_Accuracy': f"{relevance_accuracy:.1%}",
    'Mechanism_Accuracy': f"{mechanism_accuracy:.1%}",
    'Quality_Accuracy': f"{quality_accuracy:.1%}",
    'JSON_Stability': f"{stability_rate:.1%}",
    'Avg_Time_s': f"{df_relevance['time'].mean():.1f}"
}])

display(summary_df)

# Download results (works on Colab)
try:
    from google.colab import files
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"{MODEL_NAME.replace('/', '_')}_{timestamp}_results.csv"
    summary_df.to_csv(filename, index=False)
    files.download(filename)
    print(f'\n✓ Downloaded: {filename}')
except:
    print('\n(Not on Colab - results not auto-downloaded)')